***ML MODEL: CLUSTERING***
> 1) Carga de librerías/importaciones

> 2) Lectura de los datos

> 3) Creación del dataset para el algoritmo 

> 4) Preprocesado y estudio de las variables

> 5) Diferentes modelos: Kmeans, Hierarchical, DBSCAN

> 6) Comparación modelos

> 7) Conclusiones modelo elegido. Pilotos a fichar
---

# Librerías

In [1]:
# Librerias
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestNeighbors 
from sklearn.cluster import DBSCAN
from sklearn import metrics
from plotly.subplots import make_subplots

# Carga de Datos

In [2]:
# Circuitos
circuitos = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/circuits.csv")
# Resultados Constructores
resultados_constructores = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/constructor_results.csv")
# Standings Constructores
standing_constructores = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/constructor_standings.csv")
# Constructores
constructores = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/constructors.csv")
# Standings Conductores
standing_conductores = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/driver_standings.csv")
# Conductores
conductores = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/drivers.csv")
# Tiempos de Vuelta
tiempos_vuelta = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/lap_times.csv")
# Pit-Stops
pit_stops = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/pit_stops.csv")
# Histórico Qualifying
historico_clasificacion = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/qualifying.csv")
# Histórico Carreras
historico_carreras = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/races.csv")
# Historico Resultados
historico_resultados = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/results.csv")
# Histórico Temporadas
historico_temporadas = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/seasons.csv")
# Historico Carreras Sprint
historico_sprint = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/sprint_results.csv")
# Glosario Estados de Carrera (finalizado, retirado, avería, etc.)
estados_carrera = pd.read_csv("https://raw.githubusercontent.com/tomipegu/f1_visualizations/Datasets/status.csv")

# Dataset Modelo

In [3]:
# Fechas Carreras
fechas_carreras = historico_carreras.copy()[['raceId', 'date', 'year']]
fechas_carreras.rename(columns = {'date': 'race_date', 'year':'race_year'}, inplace = True)

In [4]:
# Conductores
conductores_reduced = conductores.copy()[['driverId', 'dob', 'nationality']]

In [5]:
# Resultados Constructores
resultados_constructores_reduced = resultados_constructores.copy()[['constructorId', 'raceId', 'points']]
resultados_constructores_reduced.rename(columns= {'points': 'constructor_points'}, inplace = True)

In [6]:
# Resultados Carreras
resultados_carreras_reduced = historico_resultados.copy()[['raceId', 'driverId', 'constructorId', 'points', 'grid', 'position', 'rank', 'statusId']]
resultados_carreras_reduced.rename(columns= {'points': 'driver_points', 'grid': 'start_position', 'position': 'final_position', 'rank': 'fastest_lap_rank'}, inplace = True)

In [7]:
# Status Carreras
estados_carreras_reduced = estados_carrera.copy()

# Clasificación de los posibles estados de carrera
status_conditions = [
    (estados_carreras_reduced['statusId'] == 1),
    (estados_carreras_reduced['statusId'] > 1) & (estados_carreras_reduced['statusId'] <= 4),
    (estados_carreras_reduced['statusId'] > 4)
]
status_values = ['Finished', 'Pilot Error', 'Vehicle Error']
estados_carreras_reduced['race_status'] = np.select(status_conditions, status_values)

# Eliminamos las categorías antiguas
estados_carreras_reduced = estados_carreras_reduced.drop('status', axis = 1)

In [8]:
# Dataset Modelo
dataset_reduced = conductores_reduced.merge(resultados_carreras_reduced, on='driverId', how='left') \
                                     .merge(resultados_constructores_reduced, on=['raceId', 'constructorId'], how='left') \
                                     .merge(estados_carreras_reduced, on=['statusId'], how='left') \
                                     .merge(fechas_carreras, on='raceId', how='left')

# Edad Conductores
dataset_reduced['dob'] = dataset_reduced.apply(lambda row: int(((pd.to_datetime(row['race_date'])-pd.to_datetime(row['dob'])).days/365.2)), axis=1)
dataset_reduced.rename(columns= {"dob": "age"}, inplace = True)

# Nos quedamos solo con las columnas relevantes
dataset_reduced_columns = ['driverId', 'age', 'nationality', 'raceId', 'constructorId', 'driver_points', 'start_position', 'final_position', 'fastest_lap_rank', 'constructor_points', 'race_status', 'race_year']
dataset_reduced = dataset_reduced[dataset_reduced_columns]

# Eliminamos todas las observaciones anteriores a 1997
dataset_reduced = dataset_reduced[dataset_reduced['race_year'] >= 1997]

# Preprocesado

## Tipología de las variables

In [9]:
# Comprobamos el tipo de las variables
dataset_reduced.dtypes

driverId                int64
age                     int64
nationality            object
raceId                  int64
constructorId           int64
driver_points         float64
start_position          int64
final_position         object
fastest_lap_rank       object
constructor_points    float64
race_status            object
race_year               int64
dtype: object

In [10]:
# Aquellos valores que contengan '\\N' los consideramos como nulos
dataset_reduced.loc[dataset_reduced["final_position"] == "\\N", "final_position"] = None
dataset_reduced.loc[dataset_reduced["fastest_lap_rank"] == "\\N", 'fastest_lap_rank'] = None

# Convertimos la Posicion Final y el Ranking de Vuelta Rapida a Int
dataset_reduced = dataset_reduced.copy().astype({'final_position':'float', 'fastest_lap_rank':'float'})
dataset_reduced = dataset_reduced.copy().astype({'final_position':'Int64', 'fastest_lap_rank':'Int64'})

## Valores nulos

In [11]:
# Comprobación valores nulos
dataset_reduced.isna().sum().sort_values(ascending = False)

fastest_lap_rank      2646
final_position        2483
constructor_points       2
driverId                 0
age                      0
nationality              0
raceId                   0
constructorId            0
driver_points            0
start_position           0
race_status              0
race_year                0
dtype: int64

Se estudian con más detalle estas tres variables que contienen valores nulos.

In [12]:
# Valores nulos en 'constructor_points'
dataset_reduced[dataset_reduced['constructor_points'].isna()].sort_values('age')

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_status,race_year
5418,39,35,Indian,860,164,0.0,0,<NA>,0,NaN,Vehicle Error,2012
5360,37,41,Spanish,860,164,0.0,0,<NA>,0,NaN,Vehicle Error,2012


Las únicas dos observaciones que cuentan con valores nulos en "constructor_points", corresponden con dos pilotos de la misma escudería (HRT) que no clasificaron para competir en el GP Australia 2012. Como no comenzaron la carrera, hemos decidido eliminarlos.

In [13]:
# Eliminamos aquellas observaciones cuyo valor "constructor_points" sea nulo
dataset_reduced = dataset_reduced[dataset_reduced["constructor_points"].notna()]

In [14]:
# Valores nulos en 'final_position' y 'fastest_lap_rank'
dataset_reduced[dataset_reduced['final_position'].isna() | dataset_reduced['fastest_lap_rank'].isna()]

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_status,race_year
6,1,23,British,24,1,0.0,1,<NA>,3,0.0,Pilot Error,2008
33,1,22,British,51,1,0.0,1,<NA>,19,8.0,Vehicle Error,2007
35,1,24,British,1,1,0.0,18,<NA>,13,0.0,Pilot Error,2009
46,1,24,British,12,1,0.0,12,<NA>,<NA>,3.0,Pilot Error,2009
51,1,24,British,17,1,0.0,1,<NA>,3,0.0,Vehicle Error,2009
...,...,...,...,...,...,...,...,...,...,...,...,...
25642,854,23,German,1082,210,0.0,6,<NA>,20,0.0,Vehicle Error,2022
25651,855,22,Chinese,1078,51,0.0,17,<NA>,20,6.0,Vehicle Error,2022
25652,855,22,Chinese,1079,51,0.0,15,<NA>,20,8.0,Vehicle Error,2022
25654,855,23,Chinese,1081,51,0.0,14,<NA>,17,0.0,Vehicle Error,2022


Los valores nulos en las categorías "fastest_lap_rank" y "final_position" corresponden con pilotos que no han podido completar una carrera. 

Sin embargo, en el caso de "fastest_lap_rank", es posible que el piloto haya podido dar varias vueltas al circuito y su vuelta rápida hay quedado registrada en el sistema, de modo que ese valor no sea nulo. Esta es la razón de que haya más valores nulos en una categoría que en la otra.

De modo que podamos analizar todos estos casos de forma igualitaria y sin pérdida de información, vamos a sustituir todos los valores nulos como si hubieran quedado últimos en dichas categorías

In [15]:
# Calculamos el número de pilotos por carrera
num_pilots_per_race = dataset_reduced.groupby('raceId', as_index = False)['driverId'].count()
num_pilots_per_race.rename(columns = {'driverId': 'num_pilots'}, inplace = True)

# Asumimos que todos los valores nulos en 'final_position' o 'fastest_lap_rank' equivalen a quedar últimos en esas categorías
dataset_reduced['final_position'] = dataset_reduced.apply(lambda row: num_pilots_per_race[num_pilots_per_race['raceId'] == row['raceId']]['num_pilots'].values[0] if (pd.isna(row['final_position'])) else row['final_position'], axis = 1)
dataset_reduced['fastest_lap_rank'] = dataset_reduced.apply(lambda row: num_pilots_per_race[num_pilots_per_race['raceId'] == row['raceId']]['num_pilots'].values[0] if (pd.isna(row['fastest_lap_rank'])) else row['fastest_lap_rank'], axis = 1)
dataset_reduced

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_status,race_year
0,1,23,British,18,1,10.0,1,1,2,14.0,Finished,2008
1,1,23,British,19,1,4.0,9,5,3,10.0,Finished,2008
2,1,23,British,20,1,0.0,3,13,19,4.0,Vehicle Error,2008
3,1,23,British,21,1,6.0,5,3,3,6.0,Finished,2008
4,1,23,British,22,1,8.0,3,2,2,8.0,Finished,2008
...,...,...,...,...,...,...,...,...,...,...,...,...
25655,855,23,Chinese,1082,51,4.0,10,8,7,10.0,Finished,2022
25656,855,23,Chinese,1083,51,0.0,9,20,0,0.0,Pilot Error,2022
25657,855,23,Chinese,1084,51,0.0,13,14,9,0.0,Vehicle Error,2022
25658,855,23,Chinese,1085,51,0.0,16,16,18,0.0,Vehicle Error,2022


In [16]:
dataset_reduced.isna().sum()

driverId              0
age                   0
nationality           0
raceId                0
constructorId         0
driver_points         0
start_position        0
final_position        0
fastest_lap_rank      0
constructor_points    0
race_status           0
race_year             0
dtype: int64

## Data Summary

In [17]:
# Extraemos un resumen de los datos para detectar anomalias
dataset_reduced.describe()

,driverId,age,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_year
count,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000,10055.000000
mean,277.039980,27.753555,534.594729,40.255694,3.256290,11.059175,11.945400,13.375932,6.530482,2009.726902
std,369.617856,4.873058,420.729557,67.417369,5.744052,6.235608,7.054584,7.248461,10.082795,7.327772
min,1.000000,17.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1997.000000
25%,15.000000,24.000000,122.500000,4.000000,0.000000,6.000000,6.000000,7.000000,0.000000,2004.000000
50%,35.000000,27.000000,349.000000,9.000000,0.000000,11.000000,11.000000,14.000000,2.000000,2010.000000
75%,815.000000,31.000000,958.000000,20.000000,4.000000,16.000000,20.000000,20.000000,10.000000,2016.000000
max,855.000000,43.000000,1086.000000,214.000000,50.000000,24.000000,24.000000,24.000000,66.000000,2022.000000


Observaciones:

*   **driver_points**: si la puntuación máxima por carrera es 26 (ganador de la carrera + vuelta rápida), ¿cómo es posible que el máximo registrado sea de 50?
*   **constructor_points**: si la puntuación máxima de una escudería por carrera es 44 (conductores 1º 25 puntos y 2º 18 puntos + vuelta rápida), ¿cómo es posible que el máximo registrado sea 66?
* **start_position**: ¿comó es posible que la posición inicial mínima sea 0?

Nota: Desde 2019, a parte de la puntuación máxima por posición en carrera (puesto 1º - 25 puntos), el ganador de la vuelta rápida en carrera (siempre y cuando haya acabado en el Top 10) se lleva un punto extra.

### Comprobación Posición Inicial

In [18]:
# Comprobamos los casos en los que la posición inicial es 0
dataset_reduced[dataset_reduced['start_position']==0]

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_status,race_year
1482,8,37,Finnish,983,6,0.0,0,20,0,12.0,Vehicle Error,2017
1512,8,39,Finnish,1013,51,1.0,0,10,13,1.0,Vehicle Error,2019
1522,8,39,Finnish,1023,51,0.0,0,15,13,2.0,Vehicle Error,2019
1558,8,41,Finnish,1063,51,0.0,0,18,0,0.0,Finished,2021
1563,8,42,Finnish,1071,51,0.0,0,12,5,0.0,Vehicle Error,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
25529,849,26,Canadian,1064,3,0.0,0,16,18,0.0,Vehicle Error,2021
25561,852,21,Japanese,1059,213,0.0,0,13,19,6.0,Vehicle Error,2021
25568,852,21,Japanese,1065,213,0.0,0,20,0,0.0,Vehicle Error,2021
25635,854,23,German,1075,210,0.0,0,20,0,2.0,Vehicle Error,2022


Los pilotos que tienen una posición inicial de 0, corresponde con los casos en los que hubo penalizaciones (su tiempo de clasificacion se anuló) y empezaron al final de la parilla o desde el pit-lane. 

Vamos a tratarlos a todos estos como si hubieran empezado últimos.

In [19]:
dataset_reduced['start_position'] = dataset_reduced.apply(lambda row: num_pilots_per_race[num_pilots_per_race['raceId'] == row['raceId']]['num_pilots'].values[0] if (row['start_position'] == 0) else row['start_position'], axis = 1)
dataset_reduced

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_status,race_year
0,1,23,British,18,1,10.0,1,1,2,14.0,Finished,2008
1,1,23,British,19,1,4.0,9,5,3,10.0,Finished,2008
2,1,23,British,20,1,0.0,3,13,19,4.0,Vehicle Error,2008
3,1,23,British,21,1,6.0,5,3,3,6.0,Finished,2008
4,1,23,British,22,1,8.0,3,2,2,8.0,Finished,2008
...,...,...,...,...,...,...,...,...,...,...,...,...
25655,855,23,Chinese,1082,51,4.0,10,8,7,10.0,Finished,2022
25656,855,23,Chinese,1083,51,0.0,9,20,0,0.0,Pilot Error,2022
25657,855,23,Chinese,1084,51,0.0,13,14,9,0.0,Vehicle Error,2022
25658,855,23,Chinese,1085,51,0.0,16,16,18,0.0,Vehicle Error,2022


### Comprobación Puntuación Conductores

#### Límite de puntos

In [20]:
# Comprobamos la puntuación de los conductores para aquellos casos donde se superan los limites
dataset_reduced[dataset_reduced['driver_points'] > 26].sort_values(['race_year', 'raceId'])

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,constructor_points,race_status,race_year
147,1,29,British,918,131,50.0,2,1,4,50.0,Finished,2014
2091,13,33,Brazilian,918,3,36.0,4,2,2,66.0,Finished,2014
23820,822,25,Finnish,918,3,30.0,3,3,6,66.0,Finished,2014


Hay 3 observaciones en las que el número de puntos se sale de los límites.

Esto se debe a que en la carrera de cierre de campeonato de 2014, el GP Abu Dabi 2014, se estableció un sistema de puntuaje doble. De ese modo, solo en esta carrera, las puntuaciones corresponden con el doble que en una carrera normal.

Por ello, podemos concluir que no se trata de un error en la recolección de los datos, sino un cambio en el reglamento de la competición.

#### Homogeneidad en el reparto de puntos

Con el objetivo de detectar más anomalías vamos a proceder a realizar un estudio mas detallado de los sistemas de puntuación de la F1.

In [21]:
# En vista de las anomalias, comprobamos si el resto de resultados son homogéneos.
# Suma de puntos totales otorgados por carrera
total_points_per_race = dataset_reduced.groupby(['raceId', 'race_year'], as_index = False).sum('driver_points')
total_points_per_race.rename(columns = {'driver_points': 'total_driver_points'}, inplace = True)
total_points_per_race[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
0,1,2009,39.0
1,2,2009,19.5
2,3,2009,39.0
3,4,2009,39.0
4,5,2009,39.0
...,...,...,...
468,1082,2022,102.0
469,1083,2022,102.0
470,1084,2022,102.0
471,1085,2022,102.0


Podemos empezar a observar, que el número total de puntos por carrera **no es fijo**.

In [22]:
# Posibles Variaciones en la cantidad de puntos totales otorgados por carrera
list(total_points_per_race.sort_values('total_driver_points')['total_driver_points'].unique())

[19.5, 26.0, 36.0, 39.0, 50.5, 101.0, 102.0, 202.0]

Como podemos observar, existe una gran diferencia entre la cantidad de puntos otorgados entre unas carreras y otras. Procedemos a analizar cada caso.

In [26]:
# CASO 1: Puntos Totales = 19.5
total_points_per_race[total_points_per_race['total_driver_points'] == 19.5].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
1,2,2009,19.5


Este caso corresponde con el GP de Malasia de 2009:
* La carrera se declaró finalizada tras 33 vueltas por la lluvia torrencial y la nula visibilidad, por lo que los resultados tomados son los de 2 vueltas antes de la suspensión de la carrera.
* Al no completarse el 75% de la distancia total prevista, sólo se dieron la mitad de los puntos.

Fuente: https://es.wikipedia.org/wiki/Anexo:Gran_Premio_de_Malasia_de_2009

In [25]:
# CASO 2: Puntos Totales = 26.0
total_points_per_race[total_points_per_race['total_driver_points'] == 26.0].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
206,207,1997,26.0
207,208,1997,26.0
208,209,1997,26.0
209,210,1997,26.0
210,211,1997,26.0
...,...,...,...
135,136,2002,26.0
136,137,2002,26.0
137,138,2002,26.0
138,139,2002,26.0


Este caso se corresponde con el reglamento de puntos vigente entre 1991 y 2002:

* 1º - 10 puntos
* 2º - 6 puntos
* 3º - 4 puntos
* 4º - 3 puntos
* 5º - 2 puntos
* 6º - 1 puntos

Nota: solo puntúan las primeras 6 posiciones. La vuelta rápida no puntúa.

In [27]:
# CASO 3: Puntos Totales = 36.0
total_points_per_race[total_points_per_race['total_driver_points'] == 36.0].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
78,79,2005,36.0


Este caso corresponde al GP de Estados Unidos de 2005:
* De veinte vehiculos que competieron, solo seis de ellos terminaron la carrera. El resto de competidores se retiraron tras la vuelta de formación (ni pudieron empezar la carrera) debido a preocupaciones de seguridad a causa de sus neumáticos.
* Se trata de uno de los eventos de la Fórmula 1 más polémicos de la historia de la Fórmula 1.

Fuente: https://es.wikipedia.org/wiki/Gran_Premio_de_los_Estados_Unidos_de_2005


In [28]:
# CASO 4: Puntos Totales = 39.0
total_points_per_race[total_points_per_race['total_driver_points'] == 39.0].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
107,108,2003,39.0
108,109,2003,39.0
109,110,2003,39.0
110,111,2003,39.0
111,112,2003,39.0
...,...,...,...
12,13,2009,39.0
13,14,2009,39.0
14,15,2009,39.0
15,16,2009,39.0


Este caso se corresponde con el reglamento de puntos vigente entre 2003 y 2009:

* 1º - 10 puntos
* 2º - 8 puntos
* 3º - 6 puntos
* 4º - 5 puntos
* 5º - 4 puntos
* 6º - 3 puntos
* 7º - 2 puntos
* 8º - 1 puntos

Nota: solo puntúan las primeras 8 posiciones. La vuelta rápida no puntúa.

In [29]:
# CASO 5: Puntos Totales = 50.5
total_points_per_race[total_points_per_race['total_driver_points'] == 50.5].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
450,1063,2021,50.5


Este caso corresponde con el GP de Bégica de 2021:
* La carrera se declaró finalizada tras 3 vueltas a causa de las condiciones meteorológicas.
* Al no completarse el 75% de la distancia total prevista, sólo se dieron la mitad de los puntos a partir de las posiciones obtenidas en la primera vuelta.

Fuente: https://es.wikipedia.org/wiki/Anexo:Gran_Premio_de_Bélgica_de_2021


In [30]:
# CASO 6: Puntos Totales = 101.0
total_points_per_race[total_points_per_race['total_driver_points'] == 101].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
223,337,2010,101.0
224,338,2010,101.0
225,339,2010,101.0
226,340,2010,101.0
227,341,2010,101.0
...,...,...,...
414,1024,2019,101.0
419,1029,2019,101.0
444,1057,2021,101.0
448,1061,2021,101.0


Este caso se corresponde con el reglamento de puntos desde 2010 hasta 2018:

* 1º - 25 puntos
* 2º - 18 puntos
* 3º - 15 puntos
* 4º - 12 puntos
* 5º - 10 puntos
* 6º - 8 puntos
* 7º - 6 puntos
* 8º - 4 puntos
* 9º - 2 puntos
* 10º - 1 punto

Nota: solo puntúan las primeras 10 posiciones. La vuelta rápida no puntúa.

A su vez, también se incluyen en esta categoría aquella carreras posteriores a 2018 en las que la vuelta rápida fue obtenida por un competidor que no logró acabar entre las 10 primeras posiciones.

In [31]:
# CASO 7: Puntos Totales = 102.0
total_points_per_race[total_points_per_race['total_driver_points'] == 102.0].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]

,raceId,race_year,total_driver_points
400,1010,2019,102.0
401,1011,2019,102.0
402,1012,2019,102.0
403,1013,2019,102.0
404,1014,2019,102.0
...,...,...,...
468,1082,2022,102.0
469,1083,2022,102.0
470,1084,2022,102.0
471,1085,2022,102.0


Este caso se corresponde con el reglamento de puntos vigente desde 2019 hasta la actualidad:

* 1º - 25 puntos
* 2º - 18 puntos
* 3º - 15 puntos
* 4º - 12 puntos
* 5º - 10 puntos
* 6º - 8 puntos
* 7º - 6 puntos
* 8º - 4 puntos
* 9º - 2 puntos
* 10º - 1 punto
* 1 punto Vuelta Rápida (siempre y cuando se haya terminado la carrera entre las primeras 10 posiciones).

Nota: solo puntúan las primeras 10 posiciones. La vuelta rápida puntúa.

In [32]:
# CASO 8: Puntos Totales = 202.0
total_points_per_race[total_points_per_race['total_driver_points'] == 202.0].sort_values(['race_year', 'raceId'])[['raceId', 'race_year', 'total_driver_points']]


,raceId,race_year,total_driver_points
318,918,2014,202.0


Este casco corresponde con el GP de Abu Dabi de 2014 (última carrera de la temporada):

* De forma exclusiva a esta carrera, se estableció un sistema de puntuaje doble. De ese modo, las puntuaciones corresponden con el doble que en una carrera normal.

Fuente: https://es.wikipedia.org/wiki/Anexo:Gran_Premio_de_Abu_Dabi_de_2014

**Conclusiones del análisis de puntuaciones:**

Tras el análisis de las puntuaciones, podemos concluir que no se trata de un error en la colecta de datos, si no que las diferencias corresponden con la evolución del reglamento a lo largo de los años.

Dado que nuestro objetivo es realizar un cluster de pilotos de cara a la temporada que viene, es importante que apliquemos el reglamento vigente. 

Por esa razón:
* Vamos a eliminar las dos competiciones en las que se corrió menos del 75% de la carrera, junto al polémico GP de Estados Unidos de 2005. Consideramos que estos casos pueden introducir ruido en el modelo.

* Vamos a proceder a homogeneizar todas las observaciones de acuerdo con el sistema de puntuación actual. De esta manera, podemos realizar una comparativa equilibrada en la capacidad de puntuar de todos los pilotos.

In [33]:
# raceId de aquellas carreras a eliminar
race_id_to_eliminate = [2, 79, 1063]

# Eliminamos aquellas carreras que pueden introducir ruido en el modelo
dataset_reduced = dataset_reduced[~dataset_reduced['raceId'].isin(race_id_to_eliminate)]

### Homogenización de las puntuciones

In [34]:
# Sistema de puntos de acuerdo con el reglamento vigente
# Diccionario {<posicion>: <puntos>}
reglamento_puntos = {1: 25.0,
                     2: 18.0,
                     3: 15.0,
                     4: 12.0,
                     5: 10.0,
                     6:  8.0,
                     7:  6.0,
                     8:  4.0,
                     9:  2.0,
                     10: 1.0
                     }

In [35]:
# 1] Establecemos las puntuaciones correspondientes de acuerdo a la posición obtenida al final de la carrera
dataset_reduced['driver_points'] = dataset_reduced.apply(lambda row: reglamento_puntos[row['final_position']] if ((row['final_position'] <= 10) & (row['final_position'] != None)) else 0.0, axis = 1)

# 2] Creamos una nueva columna para indicar si el piloto tiene la vuela rápida en carrera o no
dataset_reduced['vuelta_rapida'] = dataset_reduced.apply(lambda row: "yes" if row['fastest_lap_rank'] == 1 else "no", axis = 1)

# 3] Sumamos un punto a aquellos conductores que tengan la vuelta rápida y hayan quedado entre los 10 primeros puestos
dataset_reduced['driver_points'] = dataset_reduced.apply(lambda row: (row['driver_points'] + 1.0) if ((row['vuelta_rapida'] == "yes") & (row['final_position'] <= 10)) else row['driver_points'], axis = 1) 

# 4] Actualizamos la puntuación de los constrcutores de acuerdo con las nuevas puntuaciones calculadas
# Eliminamos los puntos de constructor desactualizados
dataset_reduced =  dataset_reduced.drop('constructor_points', axis=1)
# Calculamos los puntos de constructor actualizados
constructor_points_new = dataset_reduced.copy().groupby(['raceId', 'constructorId'], as_index = False)['driver_points'].sum()
constructor_points_new.rename(columns={'driver_points': 'constructor_points'}, inplace = True)
# Añadimos la nueva columna calculada al dataset original
dataset_reduced = dataset_reduced.merge(constructor_points_new, on = ['raceId', 'constructorId'], how='left')

dataset_reduced.sort_values(['raceId', 'constructorId'])

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,race_status,race_year,vuelta_rapida,constructor_points
35,1,24,British,1,1,0.0,18,20,13,Pilot Error,2009,no,0.0
1067,5,27,Finnish,1,1,0.0,12,20,20,Pilot Error,2009,no,0.0
449,2,31,German,1,2,1.0,9,10,5,Finished,2009,no,1.0
1593,9,24,Polish,1,2,0.0,4,14,2,Pilot Error,2009,no,1.0
534,3,23,German,1,3,9.0,5,6,1,Finished,2009,yes,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,854,23,German,1086,210,0.0,15,14,13,Vehicle Error,2022,no,0.0
6997,842,26,French,1086,213,0.0,20,12,14,Vehicle Error,2022,no,0.0
9926,852,22,Japanese,1086,213,0.0,16,19,16,Vehicle Error,2022,no,0.0
1031,4,41,Spanish,1086,214,4.0,6,8,18,Vehicle Error,2022,no,6.0


In [36]:
# Realizamos una nueva comprobación de las puntuaciones tras los nuevos cambios
total_points_per_race = dataset_reduced.groupby(['raceId', 'race_year'], as_index = False).sum('driver_points')
total_points_per_race.rename(columns = {'driver_points': 'total_driver_points'}, inplace = True)
total_points_per_race[['raceId', 'race_year', 'total_driver_points']].describe()

,raceId,race_year,total_driver_points
count,470.000000,470.000000,470.000000
mean,538.385106,2009.859574,101.629787
std,424.184790,7.391393,0.684247
min,1.000000,1997.000000,94.000000
25%,120.250000,2004.000000,101.000000
50%,350.500000,2010.000000,102.000000
75%,963.750000,2016.000000,102.000000
max,1086.000000,2022.000000,102.000000


Los nuevos datos parecen mucho mas coherentes y consistentes. 

Podemos observar que el número de puntos mínimos registrados en una carrera es 94. Estos valores corresponden a eventos en los que han finalizado la carrera menos de 10 competidores, ya sea por accidentes o averias.

Hemos decidido no eliminarlos, ya que en caso contario, podriamos perder información relevante sobre la fiabilidad de algunos pilotos.

### Comprobación Puntuación Constructores

In [37]:
# Comprobamos la puntuación de los constructores
dataset_reduced[dataset_reduced['constructor_points'] > 44].sort_values(['race_year', 'raceId'])

,driverId,age,nationality,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,race_status,race_year,vuelta_rapida,constructor_points


No existe ningún caso en el que la puntuación de los constructores supere los límites posibles.

Confirmamos que los errores que hubieran podido haber en esta columna se han solucionado tras modificar las puntuaciones en los apartados anteriores.

### Creamos nuevas métricas que pueden resultar relevantes

In [39]:
# Posiciones Ganadas en Carrera
  # Diferencia negativa: ganancia de posiciones
  # Diferencia positiva: pérdida de posiciones
dataset_reduced['positions_gain'] = dataset_reduced.apply(lambda row: row['final_position'] - row['start_position'], axis = 1)

# Porcentaje de los puntos de equipo obtenidos por cada conductor, sabiendo que en cada equipo hay 2 pilotos
dataset_reduced['percentage_points'] = dataset_reduced.apply(lambda row: (row['driver_points'] / row['constructor_points'] * 100) if row['constructor_points'] > 0 else 50.0 , axis = 1)

In [40]:
# Convertimos el race_status a variable numérica
  # O: Finished
  # 1: Vehicle Error
  # 2: Pilot Error
dataset_reduced.replace(['Finished', 'Vehicle Error', 'Pilot Error'],
                      [0,1,2], inplace = True)

# Clustering Model

A la hora de introducir los datos en el modelo, vamos a agrupar los datos de cada piloto por temporada. De ese modo, podemos obtener una visión global del desempeño de los pilotos cada año y mitigamos la variabilidad fruto de los diferentes circuitos, meteorología o problemas técnicos.

Para ello, vamos a agrupar los datos de acuerdo a dos métricas diferentes:
* **Media**: muy sensible a los valores atípicos, pero da una visión adecuada del desempeño medio de la temporada.
* **Mediana:** menos sensible a los valores atípicos y proporciona una visión adecuada del desempeño de un piloto en condiciones normales.

## **MEDIA**

In [41]:
# Agrupamos las métricas a nivel de temporada utilizando la media
dataset_reduced_grouped = dataset_reduced.groupby(['driverId', 'race_year'], as_index = False).mean()
dataset_reduced_grouped.sort_values(['driverId'])

,driverId,race_year,age,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,race_status,constructor_points,positions_gain,percentage_points
0,1,2007,22.000000,44.000000,1.0,15.705882,2.588235,4.117647,3.411765,0.176471,31.529412,1.529412,48.848483
15,1,2022,37.000000,1080.000000,131.0,11.153846,7.230769,5.076923,5.307692,0.076923,22.923077,-2.153846,45.237703
14,1,2021,36.000000,1061.666667,131.0,18.000000,3.095238,3.571429,3.380952,0.095238,28.428571,0.476190,65.401812
13,1,2020,35.000000,1038.562500,131.0,21.687500,1.875000,1.875000,2.625000,0.000000,35.375000,0.000000,63.480007
12,1,2019,34.000000,1020.000000,131.0,19.666667,2.333333,2.380952,3.476190,0.000000,35.190476,0.047619,59.852995
...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,852,2021,20.809524,1061.666667,213.0,1.523810,13.190476,13.142857,9.666667,0.761905,6.571429,-0.047619,32.153436
621,853,2021,22.000000,1061.666667,210.0,0.000000,18.523810,18.476190,16.047619,1.095238,0.000000,-0.047619,50.000000
623,854,2022,22.923077,1080.000000,210.0,0.923077,14.076923,14.384615,11.461538,0.769231,2.384615,0.307692,42.051282
622,854,2021,22.000000,1061.666667,210.0,0.000000,17.571429,17.000000,15.380952,1.000000,0.000000,-0.571429,50.000000


## KMeans Clustering

#### Funciones KMeans

1. Número óptimo de clusters

In [42]:
def get_optimum_cluster_number(df_k_means):

  # MÉTODO ELBOW
  wcss=[]
  for i in range(1,20):
      km=KMeans(n_clusters=i)
      km.fit(df_k_means)
      wcss.append(km.inertia_)

  # SILHOUETTE SCORE
  from sklearn.metrics import silhouette_score
  sil = []
  for k in range(2, 30):
    kmeans = KMeans(n_clusters=k, random_state=123)
    kmeans.fit(df_k_means)
    labels = kmeans.labels_
    sil.append(silhouette_score(df_k_means, labels, metric='euclidean'))

  # PLOTS
  
  fig = make_subplots(rows=1, cols=2,
                      subplot_titles=('Elbow Method', 'Silhouette Method'))

  fig.add_trace(
      go.Scatter(x=list(range(1,20)), y=wcss),
      row=1, col=1
  )

  fig.add_trace(
      go.Scatter(x=list(range(2,30)), y=sil),
      row=1, col=2
  )

  fig.update_xaxes(title_text = "Nº Clusters", row = 1, col = 1)
  fig.update_xaxes(title_text = "Nº Clusters", row = 1, col = 2)

  fig.update_yaxes(title_text = "WCSS", row = 1, col = 1)
  fig.update_yaxes(title_text = "Silhouette Score", row = 1, col = 2)

  fig.update_layout(height=700, width=1500, title_text="Optimum Number of Clusters", showlegend = False)


  fig.show()


  # OPTIMAL NUMBER OF CLUSTERS - Max(Silhouette Score)
  max_value = max(sil)
  index = sil.index(max_value)
  k = index + 2                                                # Starts on 0 the index and clusters on 2
  print("Number of optimal clusters is: " + str(k))

2. Obtención Clusters

In [43]:
def predict_clusters(n_clusters, df_k_means):

  # KMEANS - 6 Clusters
  km = KMeans(n_clusters=n_clusters, random_state = 123)

  # Fitting the input data
  km.fit(df_k_means)

  # Predicting the clusters of the input data
  cluster_labels = km.predict(df_k_means)

  return cluster_labels

#### **Modelo 1:**

In [44]:
# Se quiere observar la relación entre la edad y el desempeño de los pilotos
columns_k_means_1 = ['age', 'driver_points']
df_k_means_1 = dataset_reduced_grouped.copy()[columns_k_means_1]

In [45]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_k_means_1 = scaler.fit_transform(df_k_means_1)

In [46]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_k_means_1)


Number of optimal clusters is: 3


In [47]:
# =================
# KMEANS CLUSTERING
# =================

# Se predicen los clusters
cluster_labels1 = predict_clusters(3, df_scaled_k_means_1)

# Adding the cluster labels to the original dataframe
df_results_k_means1 = dataset_reduced_grouped.copy()
df_results_k_means1["cluster"] = cluster_labels1

In [48]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_k_means1.set_index("cluster")

# Calculating the average values for each cluster
cluster1_mean = df_results_k_means1.groupby("cluster").mean().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster1_mean.head(6)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,
0,24.28,14.00,14.19,15.14,0.80,1.99,4.34,48.19
1,28.72,4.64,6.08,9.09,0.23,13.08,24.94,53.53
2,33.09,12.82,13.51,15.06,0.73,2.56,5.98,47.51


La edad de máxima forma de los conductores corresponde con la mitad de su carrera, cuando tienen alrededor de 28 años. En esta etapa la experiencia del piloto juega muy a favor y se complementa con un estado de forma generalmente óptimo a esas edades.

#### **Modelo 2:** 

In [49]:
# Se quiere observar la habilidad de los pilotos en carrera sin tener en cuenta los puntos, en valor absoluto, que ganan
columns_k_means_2 = ['percentage_points', 'fastest_lap_rank', 'positions_gain']
df_k_means_2 = dataset_reduced_grouped.copy()[columns_k_means_2]

In [50]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_k_means_2 = scaler.fit_transform(df_k_means_2)

In [51]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_k_means_2)

Number of optimal clusters is: 4


In [52]:
# =================
# KMEANS CLUSTERING
# =================

# Se predicen los Clusters
cluster_labels2 = predict_clusters(4, df_scaled_k_means_2)

# Adding the cluster labels to the original dataframe
df_results_k_means2 = dataset_reduced_grouped.copy()
df_results_k_means2["cluster"] = cluster_labels2

In [55]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_k_means2.set_index("cluster")

# Calculating the average values for each cluster
cluster2_mean = df_results_k_means2.groupby("cluster").mean().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster2_mean.head(6)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,
0,27.13,17.32,15.71,18.29,0.89,1.15,2.36,49.30
1,28.54,9.90,13.35,20.65,0.78,4.79,9.85,48.80
2,27.43,8.54,9.13,8.11,0.43,7.44,12.80,58.88
3,28.17,10.26,11.28,9.81,0.57,4.37,12.01,36.03


#### **Modelo 3:**

In [56]:
# Se cogen otras variables diferentes a estudiar
columns_k_means_3 = ['age', 'driver_points', 'percentage_points', 'fastest_lap_rank', 'positions_gain']
df_k_means_3 = dataset_reduced_grouped.copy()[columns_k_means_3]

In [57]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_k_means_3 = scaler.fit_transform(df_k_means_3)

In [58]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_k_means_3)

Number of optimal clusters is: 4


In [59]:
# =================
# KMEANS CLUSTERING
# =================

# Se predicen los Clusters
cluster_labels3 = predict_clusters(4, df_scaled_k_means_3)

# Adding the cluster labels to the original dataframe
df_results_k_means3 = dataset_reduced_grouped.copy()
df_results_k_means3["cluster"] = cluster_labels3

In [60]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_k_means3.set_index("cluster")

# Calculating the average values for each cluster
cluster3_mean = df_results_k_means3.groupby("cluster").mean().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster3_mean.head(4)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,
0,24.49,15.65,14.85,15.48,0.84,1.17,2.85,47.44
1,27.38,5.53,6.58,5.90,0.25,11.66,21.54,56.08
2,28.83,8.64,12.33,20.94,0.70,5.95,12.10,49.36
3,33.85,12.14,12.67,12.47,0.66,2.94,7.28,45.58


## **MEDIANA**

In [62]:
# Se agrupan las métricas a nivel de temporada utilizando la mediana en lugar de la media
dataset_reduced_grouped_median = dataset_reduced.groupby(['driverId', 'race_year'], as_index = False).median()
dataset_reduced_grouped_median.sort_values(['driverId'])

,driverId,race_year,age,raceId,constructorId,driver_points,start_position,final_position,fastest_lap_rank,race_status,constructor_points,positions_gain,percentage_points
0,1,2007,22.0,44.0,1.0,18.0,2.0,2.0,2.0,0.0,32.0,0.0,45.454545
15,1,2022,37.0,1080.0,131.0,12.0,6.0,4.0,5.0,0.0,27.0,-2.0,44.444444
14,1,2021,36.0,1061.0,131.0,19.0,2.0,2.0,2.0,0.0,30.0,0.0,62.500000
13,1,2020,35.0,1038.5,131.0,25.0,1.0,1.0,2.0,0.0,39.0,0.0,59.090909
12,1,2019,34.0,1020.0,131.0,25.0,2.0,1.0,3.0,0.0,40.0,-1.0,58.139535
...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,852,2021,21.0,1061.0,213.0,0.0,14.0,13.0,10.0,1.0,6.0,0.0,0.000000
621,853,2021,22.0,1061.0,210.0,0.0,19.0,19.0,19.0,1.0,0.0,0.0,50.000000
623,854,2022,23.0,1080.0,210.0,0.0,15.0,14.0,13.0,1.0,0.0,-1.0,50.000000
622,854,2021,22.0,1061.0,210.0,0.0,18.0,18.0,17.0,1.0,0.0,-1.0,50.000000


#### **Modelo 1 mediana:**

In [63]:
# Se quiere observar la relación entre la edad y el desempeño de los pilotos
columns_k_means_4 = ['age', 'driver_points']
df_k_means_4 = dataset_reduced_grouped_median.copy()[columns_k_means_4]

In [64]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_k_means_4 = scaler.fit_transform(df_k_means_4)

In [65]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_k_means_4)

Number of optimal clusters is: 3


In [66]:
# =================
# KMEANS CLUSTERING
# =================

# Se predicen los Clusters
cluster_labels4 = predict_clusters(3, df_scaled_k_means_4)

# Adding the cluster labels to the original dataframe
df_results_k_means4 = dataset_reduced_grouped_median.copy()
df_results_k_means4["cluster"] = cluster_labels4

In [67]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_k_means4.set_index("cluster")

# Calculating the average values for each cluster
cluster4_median = df_results_k_means4.groupby("cluster").median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster4_median.head(3)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,
0,24.0,14.0,13.00,15.00,1.0,0.00,1.0,50.00
1,29.0,3.5,3.75,4.75,0.0,13.25,25.0,54.55
2,33.0,13.0,12.00,14.00,1.0,0.00,2.0,50.00


#### **Modelo 2 mediana:**

In [68]:
# Se quiere observar la habilidad de los pilotos en carrera sin tener en cuenta los puntos, en valor absoluto, que ganan
columns_k_means_5 = ['percentage_points', 'fastest_lap_rank', 'positions_gain']
df_k_means_5 = dataset_reduced_grouped_median.copy()[columns_k_means_5]

In [69]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_k_means_5 = scaler.fit_transform(df_k_means_5)

In [70]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_k_means_5)

Number of optimal clusters is: 5


In [71]:
# =================
# KMEANS CLUSTERING
# =================

# Se predicen los Clusters
cluster_labels5 = predict_clusters(5, df_scaled_k_means_5)

# Adding the cluster labels to the original dataframe
df_results_k_means5 = dataset_reduced_grouped_median.copy()
df_results_k_means5["cluster"] = cluster_labels5

In [72]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_k_means5.set_index("cluster")

# Calculating the average values for each cluster
cluster5_median = df_results_k_means5.groupby("cluster").median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster5_median.head(5)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,
0,27.0,8.00,8.00,8.0,0.0,4.0,9.5,50.00
1,28.0,12.00,20.00,22.0,1.0,0.0,1.0,50.00
2,26.5,13.25,12.25,12.5,1.0,0.0,6.0,13.49
3,27.0,17.00,15.00,20.0,1.0,0.0,0.0,50.00
4,26.0,8.50,7.50,9.5,0.0,5.0,7.0,87.50


#### **Modelo 3 mediana:**

In [73]:
# Se cogen otras variables diferentes a estudiar
columns_k_means_6 = ['age', 'driver_points', 'percentage_points', 'fastest_lap_rank', 'positions_gain']
df_k_means_6 = dataset_reduced_grouped_median.copy()[columns_k_means_6]

In [74]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_k_means_6 = scaler.fit_transform(df_k_means_6)

In [75]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_k_means_6)

Number of optimal clusters is: 7


In [76]:
# =================
# KMEANS CLUSTERING
# =================

# Se predicen los Clusters
cluster_labels6 = predict_clusters(7, df_scaled_k_means_6)

# Adding the cluster labels to the original dataframe
df_results_k_means6 = dataset_reduced_grouped_median.copy()
df_results_k_means6["cluster"] = cluster_labels6

In [77]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_k_means6.set_index("cluster")

# Calculating the average values for each cluster
cluster6_median = df_results_k_means6.groupby("cluster").median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster6_median.head(7)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,
0,25.0,16.0,14.0,16.0,1.0,0.0,0.0,50.00
1,29.0,3.0,3.5,22.0,0.0,13.5,25.0,50.00
2,28.0,11.0,20.0,22.0,1.0,0.0,1.0,50.00
3,28.0,4.0,4.0,4.0,0.0,12.0,25.0,51.14
4,34.0,13.0,12.0,12.5,1.0,0.0,1.0,50.00
5,26.0,9.0,8.0,10.0,0.0,4.0,6.0,87.50
6,26.5,14.0,12.5,13.0,1.0,0.0,6.0,7.69


## Hierarchical Clustering

Como se ha visto que la **mediana** como métrica nos proporcionaba un grado de especificidad más elevado, hemos decidido desarrollar el resto de modelos de clustering con esta medida (desechando la media). 

#### **Modelo 1 mediana:**

In [78]:
# Se quiere observar la relación entre la edad y el desempeño de los pilotos
columns_hierarchical_1 = ['age', 'driver_points']
df_hierarchical_1 = dataset_reduced_grouped_median.copy()[columns_hierarchical_1]

In [79]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_hierarchical_1 = scaler.fit_transform(df_hierarchical_1)

In [80]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_hierarchical_1)

Number of optimal clusters is: 3


In [81]:
# =======================
# HIERARCHICAL CLUSTERING
# =======================

hierarchical_cluster1 = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
labels_hierarchical1 = hierarchical_cluster1.fit_predict(df_scaled_hierarchical_1)

# Otros parámetros fueron probados tanto para el método y la métrica (cosine, l1, manhattan) obtieniendo peores resultados
linkage_data = linkage(df_scaled_hierarchical_1, method='ward', metric='euclidean')

# Se dibuja el dendrograma
fig = ff.create_dendrogram(linkage_data)
fig.update_layout(width=800, height=600)
fig.show()

In [82]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_hierarchical1 = dataset_reduced_grouped_median.copy()
df_results_hierarchical1.set_index(labels_hierarchical1)

# Calculating the average values for each cluster
cluster1_hierar = df_results_hierarchical1.groupby(labels_hierarchical1).median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster1_hierar.head(3)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
0,28.00,5.0,4.75,6.0,0.0,11.0,21.00,54.55
1,31.00,13.5,13.00,15.5,1.0,0.0,1.00,50.00
2,23.25,15.0,14.00,16.0,1.0,0.0,0.75,50.00


#### **Modelo 2 mediana:**

In [83]:
# Se quiere observar la habilidad de los pilotos en carrera sin tener en cuenta los puntos, en valor absoluto, que ganan
columns_hierarchical_2 = ['percentage_points', 'fastest_lap_rank', 'positions_gain']
df_hierarchical_2 = dataset_reduced_grouped_median.copy()[columns_hierarchical_2]

In [84]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_hierarchical_2 = scaler.fit_transform(df_hierarchical_2)

In [85]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_hierarchical_2)

Number of optimal clusters is: 5


In [88]:
# =======================
# HIERARCHICAL CLUSTERING
# =======================

hierarchical_cluster2 = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')
labels_hierarchical2 = hierarchical_cluster2.fit_predict(df_scaled_hierarchical_2)

# Otros parámetros fueron probados tanto para el método y la métrica (cosine, l1, manhattan) obtieniendo peores resultados
linkage_data2 = linkage(df_scaled_hierarchical_2, method='ward', metric='euclidean')

# Se dibuja el dendrograma
fig = ff.create_dendrogram(linkage_data2)
fig.update_layout(width=800, height=600)
fig.show()

In [89]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_hierarchical2 = dataset_reduced_grouped_median.copy()
df_results_hierarchical2.set_index(labels_hierarchical2)

# Calculating the average values for each cluster
cluster2_hierar = df_results_hierarchical2.groupby(labels_hierarchical2).median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster2_hierar.head(4)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
0,27.0,9.5,9.50,10.0,0.0,1.5,6.0,50.00
1,27.0,17.0,16.00,22.0,1.0,0.0,0.0,50.00
2,30.0,13.0,12.75,13.0,1.0,0.0,7.0,14.35
3,28.0,12.0,21.50,22.0,1.0,0.0,2.0,50.00


#### **Modelo 3 mediana:**

In [90]:
# Se cogen otras variables diferentes a estudiar
columns_hierarchical_3 = ['age', 'driver_points', 'percentage_points', 'fastest_lap_rank', 'positions_gain']
df_hierarchical_3 = dataset_reduced_grouped_median.copy()[columns_hierarchical_3]

In [91]:
# Se estandarizan los datos de entrada
scaler = StandardScaler()
df_scaled_hierarchical_3 = scaler.fit_transform(df_hierarchical_3)

In [92]:
# =========================
# Número óptimo de Clusters
# =========================
get_optimum_cluster_number(df_scaled_hierarchical_3)

Number of optimal clusters is: 7


In [93]:
# =======================
# HIERARCHICAL CLUSTERING
# =======================

hierarchical_cluster3 = AgglomerativeClustering(n_clusters=7, affinity='euclidean', linkage='ward')
labels_hierarchical3 = hierarchical_cluster3.fit_predict(df_scaled_hierarchical_3)

# Otros parámetros fueron probados tanto para el método y la métrica (cosine, l1, manhattan) obtieniendo peores resultados
linkage_data3 = linkage(df_scaled_hierarchical_3, method='ward', metric='euclidean')

# Se dibuja el dendrograma
fig = ff.create_dendrogram(linkage_data3)
fig.update_layout(width=800, height=600)
fig.show()

In [94]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_hierarchical3 = dataset_reduced_grouped_median.copy()
df_results_hierarchical3.set_index(labels_hierarchical3)

# Calculating the average values for each cluster
cluster3_hierar = df_results_hierarchical3.groupby(labels_hierarchical3).median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster3_hierar.head(4)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
0,25.0,16.0,14.0,17.0,1.0,0.0,0.0,50.00
1,29.0,4.5,4.0,4.0,0.0,12.0,22.0,54.55
2,34.0,12.0,12.0,12.0,1.0,0.0,2.0,50.00
3,26.0,11.0,21.5,22.0,1.0,0.0,1.0,50.00


## DBSCAN

#### **Modelo 1 mediana:**

In [100]:
# Se quiere observar la relación entre la edad y el desempeño de los pilotos
columns_dbscan_1 = ['age', 'driver_points']
df_dbscan_1 = dataset_reduced_grouped_median.copy()[columns_dbscan_1]

In [101]:
# Se generan los vecinos
neighb = NearestNeighbors(n_neighbors=2)
nbrs=neighb.fit(df_dbscan_1) 
distances,indices=nbrs.kneighbors(df_dbscan_1) 

In [102]:
# Se calcula la distancia de los vecinos más cercanos para dibujarla
distances = np.sort(distances, axis = 0) 
distances = distances[:, 1] 

fig = px.line(distances)
fig.update_traces(textposition="bottom right")
fig.update_layout(width=800, height=600, showlegend= False,
                  title='Nearest Neighbor Distances',
                  yaxis_title='Distance',
                  xaxis_title='Index')
fig.show()

In [103]:
# Variable eps is tend to be the value of the distance where the above graphic shows a curve. Min_samples are the values that need to be in at least each cluster (usually, 2*dimensions of dataset)
dbscan = DBSCAN(eps = 1, min_samples = 4).fit(df_dbscan_1) 
labels_dbscan1 = dbscan.labels_ 

In [99]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_dbscan1 = dataset_reduced_grouped_median.copy()
df_results_dbscan1.set_index(labels_dbscan1)

# Calculating the average values for each cluster
cluster1_dbscan = df_results_dbscan1.groupby(labels_dbscan1).median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster1_dbscan.head(10)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
-1,31.0,4.50,4.0,5.00,0.00,12.0,25.00,54.55
0,28.0,3.00,3.0,3.00,0.00,15.0,25.00,57.48
1,26.0,13.00,13.0,15.00,1.00,0.0,1.50,50.00
2,30.0,2.00,2.0,2.50,0.00,18.0,41.00,46.59
3,33.0,1.00,1.0,3.00,0.00,25.0,39.00,59.09
4,32.0,4.50,4.0,4.50,0.00,12.5,25.00,45.93
5,30.5,7.75,8.5,8.75,0.25,3.0,6.75,60.42
6,40.5,9.50,9.0,9.50,0.00,2.0,9.00,33.93


#### **Modelo 2 mediana:**

In [105]:
# Se quiere observar la habilidad de los pilotos en carrera sin tener en cuenta los puntos, en valor absoluto, que ganan
columns_dbscan_2 = ['percentage_points', 'fastest_lap_rank', 'positions_gain']
df_dbscan_2 = dataset_reduced_grouped_median.copy()[columns_dbscan_2]

In [106]:
# Se generan los vecinos
neighb = NearestNeighbors(n_neighbors=2)
nbrs=neighb.fit(df_dbscan_2) 
distances,indices=nbrs.kneighbors(df_dbscan_2) 

In [107]:
# Se calcula la distancia de los vecinos más cercanos para dibujarla
distances = np.sort(distances, axis = 0) 
distances = distances[:, 1] 

fig = px.line(distances)
fig.update_traces(textposition="bottom right")
fig.update_layout(width=800, height=600, showlegend= False,
                  title='Nearest Neighbor Distances',
                  yaxis_title='Distance',
                  xaxis_title='Index')
fig.show()

In [108]:
# Variable eps is tend to be the value of the distance where the above graphic shows a curve. Min_samples are the values that need to be in at least each cluster (usually, 2*dimensions of dataset)
dbscan = DBSCAN(eps = 3.5, min_samples = 6).fit(df_dbscan_2) 
labels_dbscan2 = dbscan.labels_ 

In [109]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_dbscan2 = dataset_reduced_grouped_median.copy()
df_results_dbscan2.set_index(labels_dbscan2)

# Calculating the average values for each cluster
cluster1_dbscan = df_results_dbscan2.groupby(labels_dbscan2).median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster1_dbscan.head(10)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
-1,26.5,11.00,10.00,13.25,0.75,1.0,7.00,35.36
0,29.0,6.00,5.50,6.00,0.00,9.0,19.00,41.43
1,27.0,13.50,13.00,15.00,1.00,0.0,1.00,50.00
2,25.0,7.75,7.75,9.00,0.00,4.5,4.75,100.00
3,32.0,10.00,9.00,11.00,0.00,2.0,9.00,25.00
4,29.0,5.00,5.50,22.00,0.00,9.0,22.25,41.86


#### **Modelo 3 mediana:**

In [110]:
# Se cogen otras variables diferentes a estudiar
columns_dbscan_3 = ['age', 'driver_points', 'percentage_points', 'fastest_lap_rank', 'positions_gain']
df_dbscan_3 = dataset_reduced_grouped_median.copy()[columns_dbscan_3]

In [111]:
# Se generan los vecinos
neighb = NearestNeighbors(n_neighbors=2)
nbrs=neighb.fit(df_dbscan_3) 
distances,indices=nbrs.kneighbors(df_dbscan_3) 

In [112]:
# Se calcula la distancia de los vecinos más cercanos para dibujarla
distances = np.sort(distances, axis = 0) 
distances = distances[:, 1] 

fig = px.line(distances)
fig.update_traces(textposition="bottom right")
fig.update_layout(width=800, height=600, showlegend= False,
                  title='Nearest Neighbor Distances',
                  yaxis_title='Distance',
                  xaxis_title='Index')
fig.show()

In [115]:
# Variable eps is tend to be the value of the distance where the above graphic shows a curve. Min_samples are the values that need to be in at least each cluster (usually, 2*dimensions of dataset)
dbscan = DBSCAN(eps = 5, min_samples = 10).fit(df_dbscan_3) 
labels_dbscan3 = dbscan.labels_ 

In [116]:
# =======
# RESULTS
# =======

# Setting the Cluster Label as index
df_results_dbscan3 = dataset_reduced_grouped_median.copy()
df_results_dbscan3.set_index(labels_dbscan3)

# Calculating the average values for each cluster
cluster1_dbscan = df_results_dbscan3.groupby(labels_dbscan3).median().round(2)[['age', 'start_position', 'final_position', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]
cluster1_dbscan.head(10)

,age,start_position,final_position,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
-1,29.0,8.00,7.50,11.0,0.0,5.0,12.0,50.00
0,26.0,15.25,14.75,17.0,1.0,0.0,0.0,50.00
1,27.0,4.00,4.00,4.0,0.0,12.5,22.0,55.88
2,29.0,3.00,4.00,3.0,0.0,12.5,28.0,41.94


## ¿CUÁL ES EL MEJOR MODELO?

Para examinar la calidad del modelo de clustering existen diversas métricas. A continuación se emplearán 3 métricas en las que no es necesario conocer los labels originales (unsupervised) como son **silhouette score, calinski y davies**. Si conocierámos sus etiquetas reales se podrían usar otras métricas como el adjusted_rand o el fowlkes_mallows. 

Sólo se necesita pasarle como argumentos las *features* y las predicciones hechas. 

In [117]:
# Evaluamos los diferentes modelos
# Features para modelos 1: 
columns_modelo_1 = ['age', 'driver_points']
features_modelo_1 = dataset_reduced_grouped.copy()[columns_modelo_1]
# Features para modelos 2: 
columns_modelo_2 = ['percentage_points', 'fastest_lap_rank', 'positions_gain']
features_modelo_2 = dataset_reduced_grouped.copy()[columns_modelo_2]
# Features para modelos 3: 
columns_modelo_3 = ['age', 'driver_points', 'percentage_points', 'fastest_lap_rank', 'positions_gain']
features_modelo_3 = dataset_reduced_grouped.copy()[columns_modelo_3]
# Features para modelos 1 con mediana: 
columns_modelo_4 = ['age', 'driver_points']
features_modelo_4 = dataset_reduced_grouped_median.copy()[columns_modelo_4]
# Features para modelos 2 con mediana: 
columns_modelo_5 = ['percentage_points', 'fastest_lap_rank', 'positions_gain']
features_modelo_5 = dataset_reduced_grouped_median.copy()[columns_modelo_5]
# Features para modelos 3 con mediana: 
columns_modelo_6 = ['age', 'driver_points', 'percentage_points', 'fastest_lap_rank', 'positions_gain']
features_modelo_6 = dataset_reduced_grouped_median.copy()[columns_modelo_6]



*   **Silhouette score**: puntuación entre [-1,1]. Cuanto más cercano a 1, mejor. Este valor es una medida de cómo de similar es un objeto a su propio cúmulo (cohesión) en comparación con otros cúmulos (separación).
*   **Calinski**: mide la ratio de varianza de cada punto comparado con los puntos en otros clusters contra la varianza comparada con los puntos de su mismo cluster. Un valor más alto del índice CH significa que los clusters son densos y están bien separados, aunque no hay un valor de corte "aceptable".
*   **Davies-Bouldin Index**: cuanto más pequeño sea, mejor. Se define como la medidad de similitud (relación entre las distancias dentro del clúster y las distancias entre los clústeres) media de cada cluster con respecto a los otros clusters. 






In [118]:
# Kmeans with mean 
silhouette_kmeans_mean_1 = metrics.silhouette_score(features_modelo_1,cluster_labels1)
calinski_kmeans_mean_1 = metrics.calinski_harabasz_score(features_modelo_1, cluster_labels1)
davies_kmeans_mean_1 = metrics.davies_bouldin_score(features_modelo_1, cluster_labels1)

silhouette_kmeans_mean_2 = metrics.silhouette_score(features_modelo_2,cluster_labels2)
calinski_kmeans_mean_2 = metrics.calinski_harabasz_score(features_modelo_2, cluster_labels2)
davies_kmeans_mean_2 = metrics.davies_bouldin_score(features_modelo_2, cluster_labels2)

silhouette_kmeans_mean_3 = metrics.silhouette_score(features_modelo_3,cluster_labels3)
calinski_kmeans_mean_3 = metrics.calinski_harabasz_score(features_modelo_3, cluster_labels3)
davies_kmeans_mean_3 = metrics.davies_bouldin_score(features_modelo_3, cluster_labels3)

In [119]:
# Kmeans with median 
silhouette_kmeans_median_1 = metrics.silhouette_score(features_modelo_4,cluster_labels4)
calinski_kmeans_median_1 = metrics.calinski_harabasz_score(features_modelo_4, cluster_labels4)
davies_kmeans_median_1 = metrics.davies_bouldin_score(features_modelo_4, cluster_labels4)

silhouette_kmeans_median_2 = metrics.silhouette_score(features_modelo_5,cluster_labels5)
calinski_kmeans_median_2 = metrics.calinski_harabasz_score(features_modelo_5, cluster_labels5)
davies_kmeans_median_2 = metrics.davies_bouldin_score(features_modelo_5, cluster_labels5)

silhouette_kmeans_median_3 = metrics.silhouette_score(features_modelo_6,cluster_labels6)
calinski_kmeans_median_3 = metrics.calinski_harabasz_score(features_modelo_6, cluster_labels6)
davies_kmeans_median_3 = metrics.davies_bouldin_score(features_modelo_6, cluster_labels6)

In [120]:
# Hierarchical models 
silhouette_hierarchical_1 = metrics.silhouette_score(features_modelo_4,labels_hierarchical1)
calinski_hierarchical_1 = metrics.calinski_harabasz_score(features_modelo_4, labels_hierarchical2)
davies_hierarchical_1 = metrics.davies_bouldin_score(features_modelo_4, labels_hierarchical3)

silhouette_hierarchical_2 = metrics.silhouette_score(features_modelo_5,labels_hierarchical1)
calinski_hierarchical_2 = metrics.calinski_harabasz_score(features_modelo_5, labels_hierarchical2)
davies_hierarchical_2 = metrics.davies_bouldin_score(features_modelo_5, labels_hierarchical3)

silhouette_hierarchical_3 = metrics.silhouette_score(features_modelo_6,labels_hierarchical1)
calinski_hierarchical_3 = metrics.calinski_harabasz_score(features_modelo_6, labels_hierarchical2)
davies_hierarchical_3 = metrics.davies_bouldin_score(features_modelo_6, labels_hierarchical3)

In [121]:
# DBSCAN models
silhouette_dbscan_1 = metrics.silhouette_score(features_modelo_4,labels_dbscan1)
calinski_dbscan_1 = metrics.calinski_harabasz_score(features_modelo_4, labels_dbscan2)
davies_dbscan_1 = metrics.davies_bouldin_score(features_modelo_4, labels_dbscan3)

silhouette_dbscan_2 = metrics.silhouette_score(features_modelo_5,labels_dbscan1)
calinski_dbscan_2 = metrics.calinski_harabasz_score(features_modelo_5, labels_dbscan2)
davies_dbscan_2 = metrics.davies_bouldin_score(features_modelo_5, labels_dbscan3)

silhouette_dbscan_3 = metrics.silhouette_score(features_modelo_6,labels_dbscan1)
calinski_dbscan_3 = metrics.calinski_harabasz_score(features_modelo_6, labels_dbscan2)
davies_dbscan_3 = metrics.davies_bouldin_score(features_modelo_6, labels_dbscan3)

In [122]:
# Se crea un dataframe para ver los resultados
data = [['1', 'Kmeans_mean', silhouette_kmeans_mean_1, calinski_kmeans_mean_1, davies_kmeans_mean_1], 
        ['2', 'Kmeans_mean', silhouette_kmeans_mean_2, calinski_kmeans_mean_2, davies_kmeans_mean_2],
        ['3', 'Kmeans_mean', silhouette_kmeans_mean_3, calinski_kmeans_mean_3, davies_kmeans_mean_3],
        ['1', 'Kmeans_median', silhouette_kmeans_median_1, calinski_kmeans_median_1, davies_kmeans_median_1],
        ['2', 'Kmeans_median', silhouette_kmeans_median_2, calinski_kmeans_median_2, davies_kmeans_median_2],
        ['3', 'Kmeans_median', silhouette_kmeans_median_3, calinski_kmeans_median_3, davies_kmeans_median_3],
        ['1', 'Hierarchical', silhouette_hierarchical_1, calinski_hierarchical_1, davies_hierarchical_1],
        ['2', 'Hierarchical', silhouette_hierarchical_2, calinski_hierarchical_2, davies_hierarchical_2], 
        ['3', 'Hierarchical', silhouette_hierarchical_3, calinski_hierarchical_3, davies_hierarchical_3],
        ['1', 'DBSCAN', silhouette_dbscan_1, calinski_dbscan_1, davies_dbscan_1],
        ['2', 'DBSCAN', silhouette_dbscan_2, calinski_dbscan_2, davies_dbscan_2], 
        ['3', 'DBSCAN', silhouette_dbscan_3, calinski_dbscan_3, davies_dbscan_3]]

columns = ['Modelo', 'Algoritmo', 'Silhouette', 'Calinski', 'Davies']
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=columns)
df

,Modelo,Algoritmo,Silhouette,Calinski,Davies
0,1,Kmeans_mean,0.475488,682.343991,0.727206
1,2,Kmeans_mean,0.212919,186.886577,1.981731
2,3,Kmeans_mean,0.147618,73.652708,2.104104
3,1,Kmeans_median,0.502236,780.670548,0.676049
4,2,Kmeans_median,0.287745,403.175392,1.205383
5,3,Kmeans_median,0.266091,225.728020,1.355408
6,1,Hierarchical,0.436460,8.860304,5.600617
7,2,Hierarchical,-0.000486,385.562314,1.873023
8,3,Hierarchical,0.113848,233.390503,1.507844
9,1,DBSCAN,-0.101502,8.496949,2.219090


A pesar de que el primer modelo ('age' & 'driver_points) tiene la máxima puntuación, no nos aporta gran cantidad de información. Es cierto que nos puede ayudar a entender a que edad alcanzan los pilotos su pico de rendimiento, pero como mucho nos serviría a modo de último recurso a la hora de diferenciar entre varios pilotos con características muy similiares.

Por esa razón, vamos a escoger el **Modelo 2 de KMeans - Mediana**. Se trata de un modelo con **5 clusters** que nos permite diferenciar entre el nivel de habilidad de los pilotos. Las variables utilizadas son ('percentage_points', 'fastest_lap_rank', 'positions_gain'). 

Estas variables buscan encontrar el equilibro entre la habilidad del piloto y las capacidades técnicas del vehículo que conducen. Al utilizar magnitudes relativas como 'positions_gain' podemos medir la capacidad personal de progresar en carrera, en vez de la capacidad del coche para llegar rápido a la meta. Además, sabiendo que los pilotos de una misma escudería llevan el mismo coche, nos parecía importante comparar el desempeño entre los pilotos con condiciones similiares ('percentage_points')

## MODELO FINAL

Tras examinar los resultados presentados en la tabla anterior, se ha podido comprobar de una forma más objetivo que el mejor modelo es el

In [130]:
best_model_df = df_results_k_means5.copy()

In [131]:
# Media de los valores de cada cluster
best_model_df.groupby('cluster').mean().round(2)[['age', 'start_position', 'final_position', 'positions_gain', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]

,age,start_position,final_position,positions_gain,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,,
0,27.93,8.07,7.96,-0.20,7.59,0.35,6.59,13.35,49.97
1,28.41,11.59,17.34,4.51,19.44,0.91,0.67,4.22,46.09
2,28.22,13.35,12.74,-0.53,13.43,0.66,0.79,7.68,14.06
3,27.17,15.15,13.89,-1.23,19.21,0.83,2.30,4.56,50.41
4,27.61,8.70,7.16,-1.45,11.30,0.20,5.97,8.27,87.36


In [132]:
# Mediana de los valores de cada cluster
best_model_df.groupby('cluster').median().round(2)[['age', 'start_position', 'final_position', 'positions_gain', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']]

,age,start_position,final_position,positions_gain,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points
cluster,,,,,,,,,
0,27.0,8.00,8.00,0.0,8.0,0.0,4.0,9.5,50.00
1,28.0,12.00,20.00,4.0,22.0,1.0,0.0,1.0,50.00
2,26.5,13.25,12.25,0.0,12.5,1.0,0.0,6.0,13.49
3,27.0,17.00,15.00,-1.0,20.0,1.0,0.0,0.0,50.00
4,26.0,8.50,7.50,-1.0,9.5,0.0,5.0,7.0,87.50


Análisis de los clusters:
* **Cluster 0:** Conformado por la élite de la Fórmula 1. Pertenece a este cluster los pilotos y las escuderías que más puntos consiguen por temporada.
* **Cluster 1:** Pilotos con poca fiabilidad ('race_status' cercano a uno). En clasificación son razonablemente rápidos (mitad de tabla), pero tienden a perder una gran cantidad de puestos en carrera, les falta consistencia. Pertenecen a este grupo las peores escuderías de la parrilla.
* **Cluster 2:** Pertenecen a este cluster los pilotos **secundarios** de las escuderías de media tabla. Se trata de pilotos con vehículos capaces de rascar puntos de forma consistente en las carreras, pero que no cosiguen estar a la altura de sus compañeros de escudería (solo representa el 14% de los puntos de sus respectivos equipos).
* **Cluster 3:** Pertenecen a este cluster los pilotos pertenecientes a escuderías claramente inferiores. Habitualmente ocupan las posiciones finales de la clasificación y no destacan en carrera. Las bajas capacidades técnicas de sus vehículos no permiten diferenciar entre la habilidad de sus pilotos.
* **Cluster 4:** Pertenen a este cluster los pilotos **principales** de las escuderías de media tabla. A pesar de no contar con los mejores coches de la parrilla, son capaces de coseguir puntos de forma consistente. Se destacan por mostrar una habilidad muy superior a sus compañeros (representan el 87% de los puntos de sus escuderías). Esto podría interpretarse como una muestra de su potencial si llevaran un coche superior. A su vez, se trata de pilotos muy fiables que cometen **muy pocos errores** ('race status' cercano a cero).

Dado que nuestra elección de pilotos para la temporada que viene debe basarse en los **pilotos disponibles**, hemos decidido extraer de cada cluster **únicamente** los pilotos presentes **en la última temporada**.

In [133]:
# Cluster de conductores de la temporada actual
df_results_k_means5[df_results_k_means5['race_year'] == 2022.0].groupby('cluster')['driverId'].unique()

cluster
0    [1, 20, 815, 822, 830, 832, 844, 847, 852, 854...
2                                             [4, 817]
3                       [807, 825, 840, 842, 848, 849]
4                                           [839, 846]
Name: driverId, dtype: object

In [134]:
# El mejor modelo es Modelo 2 Kmeans - Mediana
best_model_df = df_results_k_means5.copy()
best_model_df = best_model_df.merge(conductores[['driverId', 'code']], on = 'driverId', how = 'left')
best_model_df[best_model_df['race_year'] == 2022.0].groupby('cluster')['code'].unique()

cluster
0    [HAM, VET, PER, BOT, VER, SAI, LEC, RUS, TSU, ...
2                                           [ALO, RIC]
3                       [HUL, MAG, STR, GAS, ALB, LAT]
4                                           [OCO, NOR]
Name: code, dtype: object

In [129]:
best_model_df[(best_model_df['cluster'] == 4) & (best_model_df['race_year'] == 2022)].groupby('driverId').median().round(2)[['age', 'start_position', 'final_position', 'positions_gain', 'fastest_lap_rank', 'race_status', 'driver_points', 'constructor_points', 'percentage_points']].merge(conductores[['driverId', 'code']], on = 'driverId', how = 'left')

,driverId,age,start_position,final_position,positions_gain,fastest_lap_rank,race_status,driver_points,constructor_points,percentage_points,code
0,839,25.0,10.0,8.0,-1.0,10.0,0.0,4.0,8.0,75.0,OCO
1,846,22.0,8.0,7.0,1.0,8.0,0.0,6.0,6.0,75.0,NOR


# Conclusiones

A la hora de elegir pilotos para la temporada siguiente, vamos a basarnos en los dos siguientes criterios:
* Habilidad del piloto
* Presupuesto máximo: 20 millones de euros

En la Fórmula 1 existe una gran disparidad entre vehículos y escuderías. Por esa razón, a la hora de elegir piloto, teníamos claro que no podíamos guiarnos únicamente por medidas absolutas como podría ser la cantidad de puntos obtenidos en carrera.

Es evidente, que generalmente, los mejores pilotos van a pertenecer a las mejores escuderías, sin embargo, es innegable que existen otros conductores con gran potencial y habilidad con vehículos inferiores.

Bajo estas consideraciones, los clusters más interesantes para nuestro estudio son el **Cluster 0** y el **Cluster 4**. Sin embargo, el primero de ellos lo hemos descartado debido a dos razones:
* Nuestro presupuesto limitado (el caché de los pilotos en ese cluster es muy alto)
* La indisponibilidad de los conductores. Normalmente, los pilotos de la élite suelen tener contratos de larga duración con las escuderías. Además, siendo realistas, parece poco probable que un conductor ganador desee arriesgar su éxito con una nueva escudería.

A su vez, el Cluster 4 resulta muy interesante y cumple con todas las características que buscamos:
* 1. Pilotos consistentes y fiables, con pocos erorres.
* 2. Pilotos jovenes con una larga trayectoria por delante. Pilotos con gran capacidad de mejora y posibilidad a realizar un contrato a largo plazo.
* 3. Pilotos de gran habilidad, capaces de lograr buenos resultados a pesar de no contar con los mejores coches.
* 4. Consistentes en carrera y rápidos en clasificación: es tan importante ser un piloto rápido y conseguir un buen puesto a la salida, como saber mantener la posición y ganar puestos en carrera.

Por todo lo anterior, los pilotos elegidos para nuestra escudería (de acuerdo con el Cluster 4) serían:

* **Lando Norris**. Sueldo actual: 4,4 millones de euros
* **Esteban Ocon**. Sueldo actual: 4,4 millones de euros

Lo que nos permitiría hacerles una oferta superior a su sueldo actual que encuentren atractiva e irresistible. 